In [1]:
import tensorflow as tf 
import keras

Using TensorFlow backend.


In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as  plt 
import seaborn as sns

In [3]:
RANDOM_SEED = 42

In [4]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    zoom_range=0.2,
                                    shear_range=0.2,
                                    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)                    

In [7]:
train_set = train_datagen.flow_from_directory("dataset/Train",
                                                target_size=(128,128),
                                                batch_size=32,
                                                class_mode='binary')
val_set = val_datagen.flow_from_directory('dataset/Validation',
                                         target_size=(128,128),
                                         batch_size=3,
                                         class_mode='binary' )                                            

Found 10992 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [10]:
## CNN Model from stratch Approch
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from keras.callbacks import EarlyStopping,ModelCheckpoint
from livelossplot import PlotLossesKeras

In [14]:
model  = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(128,128,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1,activation='relu'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 42, 42, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 40, 40, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                819264    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)               